## 5.基于Xgboost分类算法的电商平台用户购买意向预测 

**1.数据预处理**

In [1]:
import pandas as pd
pd.set_option('display.max_rows', 10)
user = pd.read_csv('data/data_用户表.csv',engine='python',encoding='utf-8')
month2 = pd.read_csv('data/data_2月用户行为数据.csv',engine='python',encoding='utf-8')
month3 = pd.read_csv('data/data_3月用户行为数据.csv',engine='python',encoding='utf-8')
month4 = pd.read_csv('data/data_4月用户行为数据.csv',engine='python',encoding='utf-8')

In [2]:
user 

,用户ID,年龄,性别,用户级别,用户注册时间
0,200001,6.0,2.0,5,2016-01-26
1,200002,-1.0,0.0,1,2016-01-26
2,200003,4.0,1.0,4,2016-01-26
3,200004,-1.0,2.0,1,2016-01-26
4,200005,2.0,0.0,4,2016-01-26
...,...,...,...,...,...
105316,305317,3.0,2.0,4,2013-04-10
105317,305318,4.0,2.0,3,2016-01-26
105318,305319,3.0,0.0,3,2016-01-26
105319,305320,4.0,2.0,2,2016-01-26


In [3]:
month2

,用户ID,商品ID,时间,行为类别,商品类别,商品品牌
0,217567,22387,2016-01-31 23:59:54,1,8,30
1,217567,22387,2016-01-31 23:59:54,6,8,30
2,207684,4732,2016-01-31 23:59:59,1,6,375
3,207684,4732,2016-01-31 23:59:59,1,6,375
4,207684,4732,2016-01-31 23:59:59,1,6,375
...,...,...,...,...,...,...
11485379,278856,153390,2016-02-29 23:59:56,6,7,157
11485380,263603,114890,2016-02-29 23:59:56,1,9,640
11485381,263603,114890,2016-02-29 23:59:56,6,9,640
11485382,224475,5505,2016-02-29 23:59:57,6,8,214


In [4]:
temp_df1 = pd.DataFrame({'用户ID':['a','b','f']})
temp_df2 = pd.DataFrame({'用户ID':['a','a','e','c'],'用户行为类别':[1,1,2,3]})
temp_df3 = pd.merge(temp_df1,temp_df2)
print(temp_df1)
print(temp_df2)
print(temp_df3)

  用户ID
0    a
1    b
2    f
  用户ID  用户行为类别
0    a       1
1    a       1
2    e       2
3    c       3
  用户ID  用户行为类别
0    a       1
1    a       1


In [5]:
user_id = user['用户ID'].to_frame()

print('2月用户行为数据集中的用户是否在用户表中？',len(month2) == len(pd.merge(user_id,month2)))
print('3月用户行为数据集中的用户是否在用户表中？',len(month3) == len(pd.merge(user_id,month3)))
print('4月用户行为数据集中的用户是否在用户表中？',len(month4) == len(pd.merge(user_id,month4)))

2月用户行为数据集中的用户是否在用户表中？ True
3月用户行为数据集中的用户是否在用户表中？ True
4月用户行为数据集中的用户是否在用户表中？ True


In [64]:
month2.duplicated()

0           False
1           False
2           False
3            True
4            True
            ...  
11485379     True
11485380     True
11485381     True
11485382    False
11485383    False
Length: 11485384, dtype: bool

In [8]:
count = 0
for item in month2.duplicated():
    if item == True:
   
        count =count + 1
count

5754250

In [7]:
count = 0 
for item in month3.duplicated():
    if item == True:
        
        count =count + 1
count

13703224

In [9]:
count = 0
for item in month4.duplicated():
    if item == True:
     
        count =count + 1
count

7262521

In [10]:
IsDuplicated = month2.duplicated()
month2_isDuplicated=month2[IsDuplicated]

In [11]:
month2_isDuplicated.groupby('行为类别').count()

,用户ID,商品ID,时间,商品类别,商品品牌
行为类别,,,,,
1,2176378,2176378,2176378,2176378,2176378
2,636,636,636,636,636
3,1464,1464,1464,1464,1464
4,37,37,37,37,37
5,1981,1981,1981,1981,1981
6,3573754,3573754,3573754,3573754,3573754


In [12]:
user['用户注册时间']=pd.to_datetime(user['用户注册时间'])
user_after0415 = user.loc[user.用户注册时间  >= '2016-4-15']
user_after0415.size

135

In [13]:
month4['时间'] = pd.to_datetime(month4['时间'])
month4.loc[month4.时间 >= '2016-4-16']

,用户ID,商品ID,时间,行为类别,商品类别,商品品牌


In [14]:
user.head()

,用户ID,年龄,性别,用户级别,用户注册时间
0,200001,6.0,2.0,5,2016-01-26
1,200002,-1.0,0.0,1,2016-01-26
2,200003,4.0,1.0,4,2016-01-26
3,200004,-1.0,2.0,1,2016-01-26
4,200005,2.0,0.0,4,2016-01-26


In [15]:
month2.head()

,用户ID,商品ID,时间,行为类别,商品类别,商品品牌
0,217567,22387,2016-01-31 23:59:54,1,8,30
1,217567,22387,2016-01-31 23:59:54,6,8,30
2,207684,4732,2016-01-31 23:59:59,1,6,375
3,207684,4732,2016-01-31 23:59:59,1,6,375
4,207684,4732,2016-01-31 23:59:59,1,6,375


In [63]:
result_User_Action_ConversionRates = pd.read_csv("data/result_User_Action_ConversionRates.csv")
result_User_Action_ConversionRates

,用户ID,年龄,性别,用户级别,浏览数,加购物车数,删购物车数,购买数,收藏数,点击数,购买加购转化率,购买浏览转化率,购买点击转化率,购买收藏转化率
0,200001,6.00,2.00,5,212.00,22.00,13.00,1.00,0.00,414.00,0.05,0.00,0.00,1.00
1,200002,-1.00,0.00,1,238.00,1.00,0.00,0.00,0.00,484.00,0.00,0.00,0.00,nan
2,200003,4.00,1.00,4,221.00,4.00,1.00,0.00,1.00,420.00,0.00,0.00,0.00,0.00
3,200004,-1.00,2.00,1,52.00,0.00,0.00,0.00,0.00,61.00,nan,0.00,0.00,nan
4,200005,2.00,0.00,4,106.00,2.00,3.00,1.00,2.00,161.00,0.50,0.01,0.01,0.50
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
105316,305317,3.00,2.00,4,216.00,1.00,0.00,1.00,2.00,278.00,1.00,0.00,0.00,0.50
105317,305318,4.00,2.00,3,58.00,3.00,0.00,1.00,0.00,88.00,0.33,0.02,0.01,1.00
105318,305319,3.00,0.00,3,590.00,3.00,0.00,0.00,0.00,"1,189.00",0.00,0.00,0.00,nan
105319,305320,4.00,2.00,2,68.00,1.00,0.00,0.00,0.00,150.00,0.00,0.00,0.00,nan


In [17]:
ACTION_201602_FILE = "data/data_2月用户行为数据.csv"
ACTION_201603_FILE = "data/data_3月用户行为数据.csv"
ACTION_201604_FILE = "data/data_4月用户行为数据.csv"
USER_FILE = "data/data_用户表.csv"

In [18]:
import numpy as np
from collections import Counter

In [19]:
def add_user_type_count(group):
    
    behavior_type = group.行为类别.astype(int)
    
    type_cnt = Counter(behavior_type)
    
    group['浏览数'] = type_cnt[1]
    group['加购物车数'] = type_cnt[2]
    group['删购物车数'] = type_cnt[3]
    group['购买数'] = type_cnt[4]
    group['收藏数'] = type_cnt[5]
    group['点击数'] = type_cnt[6]
    
    return group[['用户ID', '浏览数', '加购物车数','删购物车数', '购买数', '收藏数', '点击数']]

In [20]:
month2_groupbyUserId=month2.loc[:1000, ['用户ID', '行为类别']].groupby(['用户ID'], as_index=False).apply(add_user_type_count)
month2_groupbyUserId.head()

,用户ID,浏览数,加购物车数,删购物车数,购买数,收藏数,点击数
0,217567,1,0,0,0,1,2
1,217567,1,0,0,0,1,2
2,207684,4,0,0,0,0,0
3,207684,4,0,0,0,0,0
4,207684,4,0,0,0,0,0


In [21]:
month2_groupbyUserId_dropDuplicates=month2_groupbyUserId.drop_duplicates('用户ID')
month2_groupbyUserId_dropDuplicates.head()

,用户ID,浏览数,加购物车数,删购物车数,购买数,收藏数,点击数
0,217567,1,0,0,0,1,2
2,207684,4,0,0,0,0,0
6,245959,1,0,0,0,0,3
9,272629,8,0,0,0,0,20
16,284046,2,0,0,0,0,1


In [22]:
temp_df = pd.DataFrame(data={'books':['bk1','bk1','bk1','bk2','bk2','bk3'], 'price': [12,12,12,15,15,17],'num':[2,1,1,4,2,2]})
temp_df

,books,price,num
0,bk1,12,2
1,bk1,12,1
2,bk1,12,1
3,bk2,15,4
4,bk2,15,2
5,bk3,17,2


In [23]:
temp_df.groupby('books',as_index=True).sum()

,price,num
books,,
bk1,36,4
bk2,30,6
bk3,17,2


In [24]:
temp_df.groupby('books',as_index=False).sum()

,books,price,num
0,bk1,36,4
1,bk2,30,6
2,bk3,17,2


In [25]:
def get_user_action_data(fname, chunk_size=20000):
    reader=pd.read_csv(fname,chunksize = chunk_size,engine='python',encoding='utf-8')
    chunks = []
    for chunk in reader:
        chunk= chunk[['用户ID', '行为类别']]
        chunks.append(chunk)
        
    chunks_concated = pd.concat(chunks, ignore_index=True)
    
    chunks_concated = chunks_concated.groupby(['用户ID'], as_index=False).apply(add_user_type_count)
    
    chunks_concated = chunks_concated.drop_duplicates('用户ID')
    return chunks_concated

In [26]:
def merge_action_data():
    action = []
    action.append(get_user_action_data(fname=ACTION_201602_FILE))
    action.append(get_user_action_data(fname=ACTION_201603_FILE))
    action.append(get_user_action_data(fname=ACTION_201604_FILE))
    
    action = pd.concat(action, ignore_index=True)
    
    action = action.groupby(['用户ID'], as_index=False).sum()
    
    action['购买加购转化率'] = action['购买数'] / action['加购物车数']
    action['购买浏览转化率'] = action['购买数'] / action['浏览数'] 
    action['购买点击转化率'] = action['购买数'] / action['点击数'] 
    action['购买收藏转化率'] = action['购买数'] / action['收藏数'] 
    
    action.loc[action['购买加购转化率'] > 1., '购买加购转化率'] = 1.
    action.loc[action['购买浏览转化率'] > 1., '购买浏览转化率'] = 1.
    action.loc[action['购买点击转化率'] > 1., '购买点击转化率'] = 1.
    action.loc[action['购买收藏转化率'] > 1., '购买收藏转化率'] = 1.
    
    return action

In [27]:
action_merged = merge_action_data()
action_merged

,用户ID,浏览数,加购物车数,删购物车数,购买数,收藏数,点击数,购买加购转化率,购买浏览转化率,购买点击转化率,购买收藏转化率
0,200001,212,22,13,1,0,414,0.045455,0.004717,0.002415,1.0
1,200002,238,1,0,0,0,484,0.000000,0.000000,0.000000,NaN
2,200003,221,4,1,0,1,420,0.000000,0.000000,0.000000,0.0
3,200004,52,0,0,0,0,61,NaN,0.000000,0.000000,NaN
4,200005,106,2,3,1,2,161,0.500000,0.009434,0.006211,0.5
...,...,...,...,...,...,...,...,...,...,...,...
105175,305317,216,1,0,1,2,278,1.000000,0.004630,0.003597,0.5
105176,305318,58,3,0,1,0,88,0.333333,0.017241,0.011364,1.0
105177,305319,590,3,0,0,0,1189,0.000000,0.000000,0.000000,NaN
105178,305320,68,1,0,0,0,150,0.000000,0.000000,0.000000,NaN


In [28]:
user_need = user[["用户ID", "年龄", "性别", "用户级别"]]

In [29]:
user_action=pd.merge(user_need,action_merged,on=['用户ID'],how='left')
user_action.to_csv("data/result_User_Action_ConversionRates.csv", index=False)

In [62]:
result_User_Action_ConversionRates = pd.read_csv("data/result_User_Action_ConversionRates.csv")
result_User_Action_ConversionRates

,用户ID,年龄,性别,用户级别,浏览数,加购物车数,删购物车数,购买数,收藏数,点击数,购买加购转化率,购买浏览转化率,购买点击转化率,购买收藏转化率
0,200001,6.00,2.00,5,212.00,22.00,13.00,1.00,0.00,414.00,0.05,0.00,0.00,1.00
1,200002,-1.00,0.00,1,238.00,1.00,0.00,0.00,0.00,484.00,0.00,0.00,0.00,nan
2,200003,4.00,1.00,4,221.00,4.00,1.00,0.00,1.00,420.00,0.00,0.00,0.00,0.00
3,200004,-1.00,2.00,1,52.00,0.00,0.00,0.00,0.00,61.00,nan,0.00,0.00,nan
4,200005,2.00,0.00,4,106.00,2.00,3.00,1.00,2.00,161.00,0.50,0.01,0.01,0.50
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
105316,305317,3.00,2.00,4,216.00,1.00,0.00,1.00,2.00,278.00,1.00,0.00,0.00,0.50
105317,305318,4.00,2.00,3,58.00,3.00,0.00,1.00,0.00,88.00,0.33,0.02,0.01,1.00
105318,305319,3.00,0.00,3,590.00,3.00,0.00,0.00,0.00,"1,189.00",0.00,0.00,0.00,nan
105319,305320,4.00,2.00,2,68.00,1.00,0.00,0.00,0.00,150.00,0.00,0.00,0.00,nan


In [44]:
rUACRs = pd.read_csv('data/result_User_Action_ConversionRates.csv',header=0)
pd.options.display.float_format = '{:,.2f}'.format
rUACRs.describe()

,用户ID,年龄,性别,用户级别,浏览数,加购物车数,删购物车数,购买数,收藏数,点击数,购买加购转化率,购买浏览转化率,购买点击转化率,购买收藏转化率
count,"105,321.00","105,318.00","105,318.00","105,321.00","105,180.00","105,180.00","105,180.00","105,180.00","105,180.00","105,180.00","72,129.00","105,172.00","103,197.00","45,986.00"
mean,"252,661.00",2.77,1.11,3.85,180.47,5.47,2.43,0.46,1.04,291.22,0.15,0.01,0.01,0.55
std,"30,403.70",1.67,0.96,1.07,273.44,10.62,5.60,1.05,3.44,460.03,0.27,0.02,0.07,0.47
min,"200,001.00",-1.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
25%,"226,331.00",3.00,0.00,3.00,40.00,0.00,0.00,0.00,0.00,59.00,0.00,0.00,0.00,0.00
50%,"252,661.00",3.00,2.00,4.00,94.00,2.00,0.00,0.00,0.00,148.00,0.00,0.00,0.00,1.00
75%,"278,991.00",4.00,2.00,5.00,212.00,6.00,3.00,1.00,0.00,342.00,0.17,0.00,0.00,1.00
max,"305,321.00",6.00,2.00,5.00,"7,605.00",369.00,231.00,50.00,99.00,"15,302.00",1.00,1.00,1.00,1.00


In [45]:
rUACRs[rUACRs['年龄'].isnull()]

,用户ID,年龄,性别,用户级别,浏览数,加购物车数,删购物车数,购买数,收藏数,点击数,购买加购转化率,购买浏览转化率,购买点击转化率,购买收藏转化率
34072,234073,nan,nan,1,32.00,6.00,4.00,1.00,0.00,41.00,0.17,0.03,0.02,1.00
38905,238906,nan,nan,1,171.00,3.00,2.00,2.00,3.00,464.00,0.67,0.01,0.00,0.67
67704,267705,nan,nan,1,342.00,18.00,8.00,0.00,0.00,743.00,0.00,0.00,0.00,nan


In [46]:
indices_isnull = rUACRs[rUACRs['年龄'].isnull()].index
rUACRs.drop(indices_isnull,axis=0,inplace=True)

In [47]:
no_action = rUACRs[(rUACRs['浏览数'].isnull()) & (rUACRs['加购物车数'].isnull()) & (rUACRs['删购物车数'].isnull()) & 
                   (rUACRs['购买数'].isnull()) & (rUACRs['收藏数'].isnull()) & (rUACRs['点击数'].isnull())]
no_action

,用户ID,年龄,性别,用户级别,浏览数,加购物车数,删购物车数,购买数,收藏数,点击数,购买加购转化率,购买浏览转化率,购买点击转化率,购买收藏转化率
1692,201693,-1.00,2.00,1,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
1936,201937,3.00,2.00,4,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
2273,202274,-1.00,2.00,1,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
2867,202868,3.00,2.00,3,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
3530,203531,3.00,0.00,4,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
104309,304310,3.00,2.00,3,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
104636,304637,4.00,2.00,2,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
104770,304771,3.00,0.00,5,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
104981,304982,3.00,2.00,4,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan


In [48]:
rUACRs.drop(no_action.index,axis=0,inplace=True)
print (len(rUACRs))

105177


In [49]:
no_buy = rUACRs[rUACRs['购买数']==0]
print (len(no_buy))

75694


In [50]:
rUACRs = rUACRs[rUACRs['购买数']!=0]
print (len(rUACRs))

29483


In [51]:
rUACRs.describe()

,用户ID,年龄,性别,用户级别,浏览数,加购物车数,删购物车数,购买数,收藏数,点击数,购买加购转化率,购买浏览转化率,购买点击转化率,购买收藏转化率
count,"29,483.00","29,483.00","29,483.00","29,483.00","29,483.00","29,483.00","29,483.00","29,483.00","29,483.00","29,483.00","29,483.00","29,483.00","29,483.00","29,483.00"
mean,"250,746.44",2.91,1.02,4.27,302.49,10.52,4.67,1.64,1.68,486.65,0.36,0.02,0.03,0.86
std,"29,979.68",1.49,0.96,0.81,391.53,14.30,7.57,1.41,4.58,658.67,0.32,0.04,0.14,0.29
min,"200,001.00",-1.00,0.00,2.00,1.00,0.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00,0.01
25%,"225,058.50",3.00,0.00,4.00,76.00,3.00,0.00,1.00,0.00,116.00,0.12,0.00,0.00,1.00
50%,"249,144.00",3.00,1.00,4.00,178.00,6.00,2.00,1.00,0.00,282.00,0.25,0.01,0.01,1.00
75%,"276,252.50",4.00,2.00,5.00,381.00,13.00,6.00,2.00,1.00,604.00,0.50,0.02,0.01,1.00
max,"305,318.00",6.00,2.00,5.00,"7,605.00",288.00,178.00,50.00,96.00,"15,302.00",1.00,1.00,1.00,1.00


In [52]:
indices_little_Buy = rUACRs[rUACRs['购买浏览转化率']<0.0005].index
print (len(indices_little_Buy))
rUACRs.drop(indices_little_Buy,axis=0,inplace=True)

90


In [53]:
indices_little_ConversionRates = rUACRs[rUACRs['购买点击转化率']<0.0005].index
print (len(indices_little_ConversionRates))
rUACRs.drop(indices_little_ConversionRates,axis=0,inplace=True)

323


In [54]:
rUACRs.describe()

,用户ID,年龄,性别,用户级别,浏览数,加购物车数,删购物车数,购买数,收藏数,点击数,购买加购转化率,购买浏览转化率,购买点击转化率,购买收藏转化率
count,"29,070.00","29,070.00","29,070.00","29,070.00","29,070.00","29,070.00","29,070.00","29,070.00","29,070.00","29,070.00","29,070.00","29,070.00","29,070.00","29,070.00"
mean,"250,767.10",2.91,1.03,4.27,280.26,10.14,4.46,1.64,1.59,447.11,0.36,0.02,0.03,0.87
std,"29,998.87",1.49,0.96,0.81,325.13,13.44,7.00,1.42,4.29,530.99,0.32,0.04,0.14,0.28
min,"200,001.00",-1.00,0.00,2.00,1.00,0.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00,0.02
25%,"225,036.00",3.00,0.00,4.00,75.00,3.00,0.00,1.00,0.00,114.00,0.12,0.00,0.00,1.00
50%,"249,200.50",3.00,1.00,4.00,174.00,6.00,2.00,1.00,0.00,275.00,0.25,0.01,0.01,1.00
75%,"276,284.00",4.00,2.00,5.00,366.00,13.00,6.00,2.00,1.00,585.00,0.50,0.02,0.01,1.00
max,"305,318.00",6.00,2.00,5.00,"5,007.00",288.00,158.00,50.00,69.00,"8,156.00",1.00,1.00,1.00,1.00


In [31]:
month2

,用户ID,商品ID,时间,行为类别,商品类别,商品品牌
0,217567,22387,2016-01-31 23:59:54,1,8,30
1,217567,22387,2016-01-31 23:59:54,6,8,30
2,207684,4732,2016-01-31 23:59:59,1,6,375
3,207684,4732,2016-01-31 23:59:59,1,6,375
4,207684,4732,2016-01-31 23:59:59,1,6,375
...,...,...,...,...,...,...
11485379,278856,153390,2016-02-29 23:59:56,6,7,157
11485380,263603,114890,2016-02-29 23:59:56,1,9,640
11485381,263603,114890,2016-02-29 23:59:56,6,9,640
11485382,224475,5505,2016-02-29 23:59:57,6,8,214


In [32]:
product = pd.read_csv('data/data_商品表.csv',engine='python',encoding='utf-8')
product

,商品ID,属性1,属性2,属性3,商品类别,商品品牌
0,100141,3,1,2,8,214
1,10015,2,1,1,8,124
2,100150,3,-1,-1,8,214
3,100152,3,1,-1,8,306
4,100154,3,1,1,8,489
...,...,...,...,...,...,...
24142,99844,3,1,1,8,489
24143,99853,3,2,2,8,489
24144,99859,3,2,2,8,489
24145,99862,2,2,2,8,812


In [33]:
comment = pd.read_csv('data/data_评论表.csv',engine='python',encoding='utf-8')
comment

,评论时间,商品ID,评论数量,存在差评,差评率
0,2016-02-01,100087,4,1,0.0380
1,2016-02-01,100088,2,0,0.0000
2,2016-02-01,100097,3,1,0.0870
3,2016-02-01,1001,2,0,0.0000
4,2016-02-01,100100,4,1,0.0588
...,...,...,...,...,...
558507,2016-04-15,99905,4,1,0.0345
558508,2016-04-15,99906,4,1,0.0370
558509,2016-04-15,99909,3,1,0.0526
558510,2016-04-15,9991,4,1,0.0538


In [61]:
result_Item_Action_ConversionRates = pd.read_csv('data/result_Item_Action_ConversionRates.csv',engine='python',encoding='utf-8')
result_Item_Action_ConversionRates

,商品ID,属性1,属性2,属性3,商品类别,商品品牌,浏览数,加购物车数,删购物车数,购买数,收藏数,点击数,购买加购转化率,购买浏览转化率,购买点击转化率,购买收藏转化率,评论数量,存在差评,差评率
0,100174,2,2,1,8,812,249.00,5.00,2.00,0.00,2.00,350.00,0.00,0.00,0.00,0.00,2.00,0.00,0.00
1,100310,3,2,-1,8,214,126.00,1.00,0.00,0.00,1.00,266.00,0.00,0.00,0.00,0.00,nan,nan,nan
2,100344,3,1,1,8,214,0.00,0.00,0.00,0.00,0.00,10.00,nan,nan,0.00,nan,nan,nan,nan
3,100462,3,1,2,8,214,860.00,21.00,5.00,0.00,5.00,"1,689.00",0.00,0.00,0.00,0.00,3.00,0.00,0.00
4,100466,3,1,1,8,306,4.00,0.00,0.00,0.00,0.00,0.00,nan,0.00,nan,nan,4.00,1.00,0.02
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3925,99625,3,1,1,8,214,"1,999.00",75.00,30.00,7.00,14.00,"3,416.00",0.09,0.00,0.00,0.50,3.00,0.00,0.00
3926,99639,3,1,1,8,214,1.00,0.00,0.00,0.00,0.00,0.00,nan,0.00,nan,nan,3.00,1.00,0.07
3927,99646,1,2,1,8,30,"4,335.00",146.00,61.00,9.00,21.00,"6,714.00",0.06,0.00,0.00,0.43,4.00,1.00,0.03
3928,99774,1,1,2,8,403,12.00,0.00,0.00,0.00,0.00,13.00,nan,0.00,0.00,nan,4.00,1.00,0.05


In [35]:
COMMENT_FILE = "data/data_评论表.csv"
PRODUCT_FILE = "data/data_商品表.csv"

In [36]:
def get_product_data():
    
    d_item=pd.read_csv(PRODUCT_FILE,header=0,engine='python',encoding='utf-8')
    
    return d_item

In [37]:
def add_product_type_count(group):
    behavior_type = group.行为类别.astype(int)
    
    type_cnt = Counter(behavior_type)
     
    group['浏览数'] = type_cnt[1]
    group['加购物车数'] = type_cnt[2]
    group['删购物车数'] = type_cnt[3]
    group['购买数'] = type_cnt[4]
    group['收藏数'] = type_cnt[5]
    group['点击数'] = type_cnt[6]
    
    return group[['商品ID', '浏览数', '加购物车数', '删购物车数', '购买数', '收藏数', '点击数']]

In [38]:
month2.loc[:1000, ['商品ID', '行为类别']].groupby(['商品ID'], as_index=False).apply(add_product_type_count).drop_duplicates('商品ID')

,商品ID,浏览数,加购物车数,删购物车数,购买数,收藏数,点击数
0,22387,1,0,0,0,0,1
2,4732,29,0,0,0,0,33
6,145895,1,0,0,0,0,3
9,107774,8,0,0,0,0,9
16,87598,2,0,0,0,0,1
...,...,...,...,...,...,...,...
969,83325,2,0,0,0,0,2
980,133005,2,0,0,0,0,2
981,134683,2,0,0,0,0,2
998,55092,0,0,0,0,1,0


In [39]:
def get_product_action_data(fname, chunk_size=20000):
    reader = pd.read_csv(fname, header=0, iterator=True,engine='python',encoding='utf-8')
    chunks = []
    loop = True
    while loop:
        try:
            
            chunk = reader.get_chunk(chunk_size)[["商品ID", "行为类别"]]
            chunks.append(chunk)
        except StopIteration:
            loop = False
            print("Iteration is stopped")
    
    chunks_concated = pd.concat(chunks, ignore_index=True)
    
    chunks_concated = chunks_concated.groupby(['商品ID'], as_index=False).apply(add_product_type_count)
   
    chunks_concated = chunks_concated.drop_duplicates('商品ID')
    return chunks_concated

In [40]:
def get_comment_data():
    comment = pd.read_csv(COMMENT_FILE, header=0,engine='python',encoding='utf-8')
    comment['评论时间'] = pd.to_datetime(comment['评论时间'])
    
    product_last_comment_indices = comment.groupby(['商品ID'])['评论时间'].transform(max) == comment['评论时间']
    
    comment = comment[product_last_comment_indices]
    
    return comment[['商品ID', '评论数量','存在差评', '差评率']]

In [41]:
def merge_action_data():
    action = []
    action.append(get_product_action_data(fname=ACTION_201602_FILE))
    action.append(get_product_action_data(fname=ACTION_201603_FILE))
    action.append(get_product_action_data(fname=ACTION_201604_FILE))
    
    action = pd.concat(action, ignore_index=True)
    
    action = action.groupby(['商品ID'], as_index=False).sum()
    
    action['购买加购转化率'] = action['购买数'] / action['加购物车数']
    action['购买浏览转化率'] = action['购买数'] / action['浏览数']
    action['购买点击转化率'] = action['购买数'] / action['点击数']
    action['购买收藏转化率'] = action['购买数'] / action['收藏数']
    
    action.loc[action['购买加购转化率'] > 1., '购买加购转化率'] = 1.
    action.loc[action['购买浏览转化率'] > 1., '购买浏览转化率'] = 1.
    action.loc[action['购买点击转化率'] > 1., '购买点击转化率'] = 1.
    action.loc[action['购买收藏转化率'] > 1., '购买收藏转化率'] = 1.
    
    return action

In [42]:
product = get_product_data()
action = merge_action_data()
comment = get_comment_data()

item_behavior = pd.merge( product, action, on=['商品ID'], how='left')
item_behavior = pd.merge( item_behavior, comment, on=['商品ID'], how='left')

item_behavior.to_csv("data/result_Item_Action_ConversionRates.csv", index=False)

Iteration is stopped
Iteration is stopped
Iteration is stopped


In [55]:
no_action_product = item_behavior[(item_behavior['浏览数'].isnull()) & (item_behavior['加购物车数'].isnull()) & (item_behavior['删购物车数'].isnull()) & 
                   (item_behavior['购买数'].isnull()) & (item_behavior['收藏数'].isnull()) & (item_behavior['点击数'].isnull())]
no_action_product 

,商品ID,属性1,属性2,属性3,商品类别,商品品牌,浏览数,加购物车数,删购物车数,购买数,收藏数,点击数,购买加购转化率,购买浏览转化率,购买点击转化率,购买收藏转化率,评论数量,存在差评,差评率
0,100141,3,1,2,8,214,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
1,10015,2,1,1,8,124,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
2,100150,3,-1,-1,8,214,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
3,100152,3,1,-1,8,306,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,3.00,0.00,0.00
4,100154,3,1,1,8,489,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24142,99844,3,1,1,8,489,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
24143,99853,3,2,2,8,489,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
24144,99859,3,2,2,8,489,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
24145,99862,2,2,2,8,812,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan


In [56]:
item_behavior.drop(no_action_product.index,axis=0,inplace=True)
print (len(item_behavior))

3930


In [57]:
item_behavior.describe()

,商品ID,属性1,属性2,属性3,商品类别,商品品牌,浏览数,加购物车数,删购物车数,购买数,收藏数,点击数,购买加购转化率,购买浏览转化率,购买点击转化率,购买收藏转化率,评论数量,存在差评,差评率
count,"3,930.00","3,930.00","3,930.00","3,930.00","3,930.00","3,930.00","3,930.00","3,930.00","3,930.00","3,930.00","3,930.00","3,930.00","2,221.00","3,901.00","3,684.00","2,013.00","2,817.00","2,817.00","2,817.00"
mean,"85,326.94",1.99,1.02,1.30,8.00,447.12,"1,727.08",54.32,21.33,3.38,10.68,"2,795.58",0.04,0.00,0.00,0.16,2.98,0.58,0.04
std,"49,296.10",1.15,0.91,0.91,0.00,252.53,"7,965.40",286.00,106.20,21.72,50.00,"12,659.80",0.09,0.00,0.00,0.26,1.04,0.49,0.10
min,18.00,-1.00,-1.00,-1.00,8.00,30.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00
25%,"42,680.50",1.00,1.00,1.00,8.00,214.00,19.00,0.00,0.00,0.00,0.00,30.00,0.00,0.00,0.00,0.00,2.00,0.00,0.00
50%,"85,920.50",2.00,1.00,2.00,8.00,403.00,148.00,1.00,1.00,0.00,1.00,247.00,0.00,0.00,0.00,0.00,3.00,1.00,0.02
75%,"127,364.00",3.00,2.00,2.00,8.00,658.00,658.00,12.00,5.00,0.00,4.00,"1,078.75",0.05,0.00,0.00,0.25,4.00,1.00,0.05
max,"171,182.00",3.00,2.00,2.00,8.00,916.00,"194,920.00","6,296.00","2,258.00",691.00,"1,205.00","312,005.00",1.00,0.11,0.06,1.00,4.00,1.00,1.00


In [60]:
item_behavior.to_csv("data/result_Item_Action_ConversionRates.csv", index=False)
item_behavior

,商品ID,属性1,属性2,属性3,商品类别,商品品牌,浏览数,加购物车数,删购物车数,购买数,收藏数,点击数,购买加购转化率,购买浏览转化率,购买点击转化率,购买收藏转化率,评论数量,存在差评,差评率
7,100174,2,2,1,8,812,249.00,5.00,2.00,0.00,2.00,350.00,0.00,0.00,0.00,0.00,2.00,0.00,0.00
20,100310,3,2,-1,8,214,126.00,1.00,0.00,0.00,1.00,266.00,0.00,0.00,0.00,0.00,nan,nan,nan
25,100344,3,1,1,8,214,0.00,0.00,0.00,0.00,0.00,10.00,nan,nan,0.00,nan,nan,nan,nan
49,100462,3,1,2,8,214,860.00,21.00,5.00,0.00,5.00,"1,689.00",0.00,0.00,0.00,0.00,3.00,0.00,0.00
51,100466,3,1,1,8,306,4.00,0.00,0.00,0.00,0.00,0.00,nan,0.00,nan,nan,4.00,1.00,0.02
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24118,99625,3,1,1,8,214,"1,999.00",75.00,30.00,7.00,14.00,"3,416.00",0.09,0.00,0.00,0.50,3.00,0.00,0.00
24120,99639,3,1,1,8,214,1.00,0.00,0.00,0.00,0.00,0.00,nan,0.00,nan,nan,3.00,1.00,0.07
24121,99646,1,2,1,8,30,"4,335.00",146.00,61.00,9.00,21.00,"6,714.00",0.06,0.00,0.00,0.43,4.00,1.00,0.03
24134,99774,1,1,2,8,403,12.00,0.00,0.00,0.00,0.00,13.00,nan,0.00,0.00,nan,4.00,1.00,0.05
